## Regression Example Using TensorFlow 2
### from Geron, 2nd Edition, chapter 10

In [1]:
import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.4.1'

In [2]:
keras.__version__

'2.4.0'

In [3]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

print(housing.data)
print(housing.target)
print(len(housing.data[0]))
print(len(X_train))
print(len(y_train))
X_train.shape[1:]

[[   8.3252       41.            6.98412698 ...    2.55555556
    37.88       -122.23      ]
 [   8.3014       21.            6.23813708 ...    2.10984183
    37.86       -122.22      ]
 [   7.2574       52.            8.28813559 ...    2.80225989
    37.85       -122.24      ]
 ...
 [   1.7          17.            5.20554273 ...    2.3256351
    39.43       -121.22      ]
 [   1.8672       18.            5.32951289 ...    2.12320917
    39.43       -121.32      ]
 [   2.3886       16.            5.25471698 ...    2.61698113
    39.37       -121.24      ]]
[4.526 3.585 3.521 ... 0.923 0.847 0.894]
8
11610
11610


(8,)

### We have 11,610 records of X and y to train from.  The X contains 8 attributes, and the y are single values.

In [4]:
model = keras.models.Sequential([
    keras.layers.Dense(8, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

In [5]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss  = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [6]:
model.compile(loss=huber_fn, optimizer="sgd")

In [7]:
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 0.8859 - val_loss: 0.3581
Epoch 2/20
363/363 [==============================] - 0s 967us/step - loss: 0.3286 - val_loss: 0.2928
Epoch 3/20
363/363 [==============================] - 0s 956us/step - loss: 0.2674 - val_loss: 0.2692
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.2520 - val_loss: 0.2559
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 0.2366 - val_loss: 0.2459
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 0.2374 - val_loss: 0.2393
Epoch 7/20
363/363 [==============================] - 0s 946us/step - loss: 0.2286 - val_loss: 0.2360
Epoch 8/20
363/363 [==============================] - 0s 920us/step - loss: 0.2116 - val_loss: 0.2324
Epoch 9/20
363/363 [==============================] - 0s 913us/step - loss: 0.2115 - val_loss: 0.2310
Epoch 10/20
363/363 [==============================] - 0s 932us/step - loss: 0.2174 - val_

In [8]:
mse_test = model.evaluate(X_test, y_test)
mse_test

162/162 [==============================] - 0s 644us/step - loss: 0.1917


0.19171291589736938

In [9]:
X_new1 = X_test[:3] # pretend these are new instances
y_pred1 = model.predict(X_new1)
y_pred1

array([[1.480097 ],
       [1.2040054],
       [3.8797677]], dtype=float32)

In [10]:
X_new2 = X_test[:3] # pretend these are new instances
y_pred2 = model.predict(X_new2)
y_pred2

array([[1.480097 ],
       [1.2040054],
       [3.8797677]], dtype=float32)

### Now set up TensorBoard, and train again

In [11]:
import os
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir() # e.g., './my_logs/run_2019_06_07-15_15_22'

In [12]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid),
                    callbacks=[tensorboard_cb])

Epoch 1/20
363/363 [==============================] - 0s 1ms/step - loss: 0.1976 - val_loss: 0.2156
Epoch 2/20
363/363 [==============================] - 0s 968us/step - loss: 0.1976 - val_loss: 0.2149
Epoch 3/20
363/363 [==============================] - 0s 948us/step - loss: 0.1972 - val_loss: 0.2148
Epoch 4/20
363/363 [==============================] - 0s 961us/step - loss: 0.1963 - val_loss: 0.2150
Epoch 5/20
363/363 [==============================] - 0s 967us/step - loss: 0.1959 - val_loss: 0.2144
Epoch 6/20
363/363 [==============================] - 0s 921us/step - loss: 0.1955 - val_loss: 0.2132
Epoch 7/20
363/363 [==============================] - 0s 942us/step - loss: 0.1952 - val_loss: 0.2127
Epoch 8/20
363/363 [==============================] - 0s 975us/step - loss: 0.1939 - val_loss: 0.2121
Epoch 9/20
363/363 [==============================] - 0s 961us/step - loss: 0.1948 - val_loss: 0.2118
Epoch 10/20
363/363 [==============================] - 0s 990us/step - loss: 0.1941 

In [13]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006